In [1]:
import gpt_2_simple as gpt2
import os
from datetime import datetime 
import tensorflow as tf
from google.cloud import storage
import pandas as pd
import numpy as np

In [2]:
files = [x for x in os.listdir('/home/jupyter/NLP_disco_polo_generator/samples') if 'sample' in x]

In [3]:
texts = []
for file in files:
    with open('/home/jupyter/NLP_disco_polo_generator/samples/' + file) as f:
        texts.append(f.read())

In [4]:
b = int(len(texts)*0.8)
texts_train = texts[:b]
texts_test = texts[b::]

In [5]:
pd.DataFrame(texts_train).to_csv('generated_train.csv', index = False)

In [6]:
pd.DataFrame(texts_test).to_csv('generated_test.csv', index = False)

In [7]:
train_not_generated = pd.read_csv('train.csv')
train_not_generated.columns = ['text']
train_not_generated['label'] = 0

In [8]:
train_lstm = pd.concat([pd.DataFrame({'text':texts_train, 'label':[1]*len(texts_train)}), train_not_generated])

In [9]:
test_not_generated = pd.read_csv('test.csv')
test_not_generated.columns = ['text']
test_not_generated['label'] = 0

In [10]:
test_lstm = pd.concat([pd.DataFrame({'text':texts_test, 'label':[1]*len(texts_test)}), test_not_generated])

In [11]:
train_lstm.to_csv('/home/jupyter/NLP_disco_polo_generator/MS/lstm_train.csv', index = False)
test_lstm.to_csv('/home/jupyter/NLP_disco_polo_generator/MS/lstm_test.csv', index = False)

In [12]:
from datasets import load_dataset 
import tensorflow as tf

In [51]:
dataset = load_dataset("csv", data_files={"train": '/home/jupyter/NLP_disco_polo_generator/MS/lstm_train.csv',
                                     "test": '/home/jupyter/NLP_disco_polo_generator/MS/lstm_test.csv'}) 

Using custom data configuration default-051d85e44f2ae552
Reusing dataset csv (/home/jupyter/.cache/huggingface/datasets/csv/default-051d85e44f2ae552/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [15]:
train_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(train_lstm['text'].values, tf.float32),
            tf.cast(train_lstm['label'].values, tf.int32)
        )
    )
)

In [16]:
test_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(test_lstm['text'].values, tf.float32),
            tf.cast(test_lstm['label'].values, tf.int32)
        )
    )
)

In [17]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [20]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

RuntimeError: `adapt` is only supported in tensorflow v2.